# Multi-Symbol Portfolio Backtesting

Learn how to backtest strategies across multiple assets.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/johan-gras/mantis/blob/main/notebooks/multi_symbol.ipynb)

In [ ]:
!pip install mantis-bt -q

In [ ]:
import mantis as mt
import numpy as np

## Load Multiple Symbols

Mantis bundles sample data for AAPL, SPY, and BTC.

In [ ]:
# Load sample data for multiple symbols
aapl = mt.load_sample("AAPL")
spy = mt.load_sample("SPY")
btc = mt.load_sample("BTC")

print(f"AAPL: {len(aapl['bars'])} bars")
print(f"SPY: {len(spy['bars'])} bars")
print(f"BTC: {len(btc['bars'])} bars")

## Single-Symbol Backtests

Run backtests on each symbol separately.

In [ ]:
# Run backtests with SMA crossover strategy
results_aapl = mt.backtest(aapl, strategy="sma-crossover")
results_spy = mt.backtest(spy, strategy="sma-crossover")
results_btc = mt.backtest(btc, strategy="sma-crossover")

print("AAPL:")
print(results_aapl)
print("\nSPY:")
print(results_spy)
print("\nBTC:")
print(results_btc)

## Compare Strategies Across Symbols

Use `mt.compare()` to view side-by-side metrics.

In [ ]:
comparison = mt.compare(
    [results_aapl, results_spy, results_btc],
    names=["AAPL", "SPY", "BTC"]
)
print(comparison)

In [ ]:
# Plot comparison (overlaid equity curves)
comparison.plot()

## Aggregate Portfolio Metrics

For portfolio-level analysis, run individual backtests and combine their equity curves.

In [ ]:
# Get equity curves from each backtest
equity_aapl = results_aapl.equity_curve
equity_spy = results_spy.equity_curve
equity_btc = results_btc.equity_curve

# Combine into portfolio (equal-weighted for simplicity)
# Note: Actual production code would align dates and handle missing data
min_len = min(len(equity_aapl), len(equity_spy), len(equity_btc))
portfolio_equity = (
    equity_aapl[:min_len] + 
    equity_spy[:min_len] + 
    equity_btc[:min_len]
) / 3

# Calculate portfolio returns
portfolio_returns = np.diff(portfolio_equity) / portfolio_equity[:-1]
portfolio_sharpe = np.mean(portfolio_returns) / np.std(portfolio_returns) * np.sqrt(252)
portfolio_max_dd = np.min(portfolio_equity / np.maximum.accumulate(portfolio_equity) - 1)

print(f"Portfolio Sharpe: {portfolio_sharpe:.2f}")
print(f"Portfolio Max Drawdown: {portfolio_max_dd:.2%}")

## Loading Your Own Data

To use your own data, load multiple CSV files.

In [ ]:
# Example with local files (uncomment to use)
# aapl_data = mt.load("/path/to/AAPL.csv")
# msft_data = mt.load("/path/to/MSFT.csv")
# googl_data = mt.load("/path/to/GOOGL.csv")

## Load Directory of Files

Load all CSV files from a directory at once.

In [ ]:
# Example with directory (uncomment to use)
# data_dict = mt.load_dir("/path/to/data/")
# for symbol, data in data_dict.items():
#     print(f"{symbol}: {len(data['bars'])} bars")

## Using Multiple Files with Glob Patterns

In [ ]:
# Example with glob pattern (uncomment to use)
# data_dict = mt.load_multi("/path/to/data/*.csv")

## Different Strategies Per Symbol

You can run different strategies for each symbol.

In [ ]:
# Momentum for stocks, mean reversion for BTC
results_aapl_mom = mt.backtest(aapl, strategy="momentum")
results_spy_mom = mt.backtest(spy, strategy="momentum")
results_btc_mr = mt.backtest(btc, strategy="mean-reversion")

print("AAPL (Momentum):")
print(f"  Sharpe: {results_aapl_mom.metrics()['sharpe']:.2f}")
print("SPY (Momentum):")
print(f"  Sharpe: {results_spy_mom.metrics()['sharpe']:.2f}")
print("BTC (Mean Reversion):")
print(f"  Sharpe: {results_btc_mr.metrics()['sharpe']:.2f}")

## Validate Multi-Symbol Results

In [ ]:
# Validate each symbol's results
for name, result in [("AAPL", results_aapl), ("SPY", results_spy), ("BTC", results_btc)]:
    validation = result.validate()
    print(f"{name}: {validation.verdict} (OOS/IS: {validation.efficiency_ratio:.0%})")

## Crypto vs Traditional Hours

Mantis automatically detects data frequency and adjusts annualization factors.

- Stock data: Uses trading hours (~252 days/year)
- Crypto data: Uses 24/7 trading (365 days/year)

In [ ]:
# Force 24-hour trading for crypto
results_btc_24h = mt.backtest(btc, strategy="sma-crossover", trading_hours_24=True)
print(f"BTC (24h): Sharpe = {results_btc_24h.metrics()['sharpe']:.2f}")

# Compare with default (auto-detected)
print(f"BTC (auto): Sharpe = {results_btc.metrics()['sharpe']:.2f}")

## Key Takeaways

1. **Load sample data** with `mt.load_sample()` for AAPL, SPY, BTC
2. **Compare results** with `mt.compare()` for side-by-side analysis
3. **Aggregate equity curves** for portfolio-level metrics
4. **Load your data** with `mt.load()`, `mt.load_dir()`, or `mt.load_multi()`
5. **Auto-detect frequency** ensures correct annualization for stocks vs crypto

## Next Steps

- [Quick Start](https://colab.research.google.com/github/johan-gras/mantis/blob/main/notebooks/quickstart.ipynb) - Basic usage
- [Validation](https://colab.research.google.com/github/johan-gras/mantis/blob/main/notebooks/validation.ipynb) - Avoid overfitting